# *Module download*

In [ ]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install vosk
!pip install pydub
!pip install jiwer
!pip install huggingface_hub
!pip install soundfile
!pip install datasets pyarrow audioread pydub --quiet
!pip install --upgrade --quiet speechbrain
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install ctranslate2 faster-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

# *Module import*


In [ ]:
# Core machine learning & audio processing
import torch
import torchaudio
import librosa
import soundfile as sf

# Hugging Face transformers and evaluation
from transformers import pipeline
import datasets
from huggingface_hub import snapshot_download

from datasets import Audio
import os

# File handling (Google Colab)
from google.colab import files

# Visualization
import matplotlib.pyplot as plt

# General utilities
import os
import numpy as np
from vosk import Model, KaldiRecognizer
#  from speechbrain.pretrained import EncoderDecoderASR
import wave
import json
from pydub import AudioSegment
from jiwer import wer, cer
import tempfile
from datasets import load_dataset, Audio
from speechbrain.pretrained import EncoderDecoderASR
import pandas as pd
import time
from faster_whisper import WhisperModel

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover
<ipython-input-3-216684b9cac2>:32: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderDecoderASR


Downloading and Unzipping Linagora asr Model

In [ ]:
!curl -L https://huggingface.co/linagora/linto-asr-ar-tn-0.1/resolve/main/vosk-model.zip --output vosk-model.zip
!unzip vosk-model.zip -d linto-asr-ar-tn-0.1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1176  100  1176    0     0   6412      0 --:--:-- --:--:-- --:--:--  6426
100  516M  100  516M    0     0   160M      0  0:00:03  0:00:03 --:--:--  186M
Archive:  vosk-model.zip
   creating: linto-asr-ar-tn-0.1/vosk-model/
   creating: linto-asr-ar-tn-0.1/vosk-model/graph/
  inflating: linto-asr-ar-tn-0.1/vosk-model/graph/phones.txt  
  inflating: linto-asr-ar-tn-0.1/vosk-model/graph/num_pdfs  
  inflating: linto-asr-ar-tn-0.1/vosk-model/graph/disambig_tid.int  
  inflating: linto-asr-ar-tn-0.1/vosk-model/graph/HCLG.fst  
   creating: linto-asr-ar-tn-0.1/vosk-model/graph/phones/
  inflating: linto-asr-ar-tn-0.1/vosk-model/graph/phones/align_lexicon.int  
  inflating: linto-asr-ar-tn-0.1/vosk-model/graph/phones/optional_silence.csl  
  inflating: linto-asr-ar-tn-0.1/vosk-model/graph/phones/word_boundary.int  
  inflating: lint

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
ds = load_dataset("linagora/linto-dataset-audio-ar-tn", split="test")
ds = ds.cast_column("audio", Audio(sampling_rate=16000))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/23.2k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

train-00000-of-00002.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/76.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20895 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/799 [00:00<?, ? examples/s]

In [ ]:
print(len(ds))
print(ds[0].keys())       # e.g. {'audio', 'transcript', ...}
print(ds[0]['audio'])     # {'array': array([...]), 'path': '/root/.cache/.../0.wav', 'sampling_rate':16000}
print(ds[0]['transcript'])
audio_files = ["Recording.wav",]

799
dict_keys(['audio_id', 'audio', 'segments', 'transcript'])
{'path': '001_chunk_288.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}
ويرحمني وإياكم في الدنيا والآخرة


In [ ]:
results = [] # for storing the results

In [ ]:
def test_hf_model(model_name_or_path, audio_path):
    pipe = pipeline("automatic-speech-recognition", device=device, model=model_name_or_path, generate_kwargs={"language": "arabic"}, return_timestamps=True)
    result = pipe(audio_path)
    return result["text"]


In [ ]:
def convert_audio_to_mono_pcm_wav(audio_path):

    audio = AudioSegment.from_file(audio_path)

    audio = audio.set_channels(1).set_sample_width(2).set_frame_rate(16000)

    temp_wav = tempfile.NamedTemporaryFile(delete=False, suffix='.wav')
    audio.export(temp_wav.name, format="wav")

    return temp_wav.name

In [ ]:
def add_result(results_list, sample_id, model_name, reference, output, error, rtf):
    results_list.append({
        "sample_id": sample_id,
        "model": model_name,
        "transcript": reference,
        "output": output,
        "wer": round(error, 4) if error is not None else None,
        "rtf": round(rtf, 4) if rtf is not None else None
    })


In [ ]:
def test_vosk_model(model_dir, audio_path):
    model = Model(model_dir)
    temp_audio_path = convert_audio_to_mono_pcm_wav(audio_path)
    with wave.open(temp_audio_path, "rb") as wf:
        if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
            raise ValueError("Audio must be mono PCM WAV.")
        rec = KaldiRecognizer(model, wf.getframerate())
        rec.AcceptWaveform(wf.readframes(wf.getnframes()))
        result = json.loads(rec.FinalResult())
    os.remove(temp_audio_path)
    return result["text"]

In [ ]:
def test_loop_hf_model(name,model_id, dstaset):
  all_refs = []
  all_hyps = []
  for sample in ds:
        reference = sample.get("transcript", "")

        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmpfile:
            sf.write(tmpfile.name, sample["audio"]["array"], samplerate=16000)
            audio_path = tmpfile.name  # Save path for use outside 'with'

        try:
            audio_duration = librosa.get_duration(path=audio_path)
            # Measure processing time
            start_time = time.time()
            hyp = test_hf_model(model_id, audio_path)
            end_time = time.time()
            error=wer(reference, hyp)
            processing_time = end_time - start_time
            rtf = processing_time / audio_duration
            print(f"********  Ref: {reference}")
            print(f"  {hyp}   (WER={error:.3f})")
            print(f"  Real-Time Factor (RTF): {rtf:.2f}")
            add_result(results,sample["audio_id"], name, reference, hyp, error,rtf)
            all_refs.append(reference)
            all_hyps.append(hyp)
        except Exception as e:
            print(f"  failed: {e}")
        finally:
            # Clean up the temporary file no matter what
            if os.path.exists(audio_path):
                os.remove(audio_path)
  full_ref = " ".join(all_refs)
  full_hyp = " ".join(all_hyps)
  corpus_wer = wer(full_ref, full_hyp)
  print(f"\n Final Corpus-level WER: {corpus_wer:.3f}")

In [ ]:
def test_loop_vosk_model(name,model_dir, audio_path):
  all_refs = []
  all_hyps = []
  for sample in ds:
        reference = sample.get("transcript", "")
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmpfile:
            sf.write(tmpfile.name, sample["audio"]["array"], samplerate=16000)
            audio_path = tmpfile.name
        try:
            audio_duration = librosa.get_duration(path=audio_path)
            # Measure processing time
            start_time = time.time()
            hyp = test_vosk_model(model_dir, audio_path)
            end_time = time.time()
            error=wer(reference, hyp)
            processing_time = end_time - start_time
            rtf = processing_time / audio_duration
            print(f"********  Ref: {reference}")
            print(f"  {hyp}   (WER={error:.3f})")
            print(f"  Real-Time Factor (RTF): {rtf:.2f}")
            add_result(results,sample["audio_id"], name, reference, hyp, error, rtf)
            all_refs.append(reference)
            all_hyps.append(hyp)
        except Exception as e:
            print(f"  failed: {e}")
        finally:

            if os.path.exists(audio_path):
                os.remove(audio_path)
  full_ref = " ".join(all_refs)
  full_hyp = " ".join(all_hyps)
  corpus_wer = wer(full_ref, full_hyp)
  print(f"\n Final Corpus-level WER: {corpus_wer:.3f}")

In [ ]:
hf_models = {
    "Tarjamni=": "mahdi02ch/whisper-tunisian-dialect",
    #"Whisper-large-v3-turbo": "openai/whisper-large-v3-v3-turbo",
    #"Whisper-large-v3": "openai/whisper-large-v3",
    #"Tarjamni=": "mahdi02ch/whisper-tunisian-dialect",
}

vosk_models = {
    "Linto_-asr-ar-tn-0.1": "/content/linto-asr-ar-tn-0.1/vosk-model"
}


In [ ]:
for name, model_id in hf_models.items():
  print(f"\n Evaluating Hugging Face model: {name} ")
  test_loop_hf_model(name,model_id, ds)

In [ ]:
df = pd.DataFrame(results)
print(df)
df.to_csv("results.csv", index=False, encoding='utf-8-sig')
files.download("results.csv")

In [ ]:
transcript_list=df['transcript'].tolist()
output_list=df['output'].tolist()
print(transcript_list[:5])
print(output_list[:5])

In [ ]:
transcript=" ".join(transcript_list)
output = " ".join(str(item) for item in output_list)
Wer=wer(transcript,output)
cer=cer(transcript,output)
print(Wer)

In [ ]:
average_rtf = df['rtf'].mean()

print("Average RTF:", average_rtf)

In [ ]:
df = pd.DataFrame(results)
print(df)
df.to_csv("results.csv", index=False, encoding='utf-8-sig')
files.download("results.csv")

transcript_list=df['transcript'].tolist()
output_list=df['output'].tolist()
print(transcript_list[:5])
print(output_list[:5])

transcript=" ".join(transcript_list)
output = " ".join(str(item) for item in output_list)
Wer=wer(transcript,output)
cer=cer(transcript,output)
print(Wer)

average_rtf = df['rtf'].mean()

print("Average RTF:", average_rtf)